In [1]:
# Set the folder paths and dataset CSV path (customize these according to your setup)
image_folder = 'C:\\Users\\Jeffan\\Downloads\\MVSA-multiple\\MVSA\\data'
dataset_csv = 'C:\\Users\\Jeffan\\Documents\\GitHub\\CS16-1-Sentiment-Analysis\\Data Image\\output_multi.csv'
train_folder = 'C:\\Users\\Jeffan\\Downloads\\MVSA-multiple\\MVSA\\train'
val_folder = 'C:\\Users\\Jeffan\\Downloads\\MVSA-multiple\\MVSA\\val'
test_folder = 'C:\\Users\\Jeffan\\Downloads\\MVSA-multiple\\MVSA\\test'

In [5]:
ids = []
texts = []
images = []
with open('C:\\Users\\Jeffan\\Downloads\\MVSA-multiple\\MVSA\\labelResultAll.txt', 'r', encoding='ISO-8859-1') as file:
    for line in file:
        text_list = line.replace('\n','').split('	')
        text_label_result = None
        text_label_1 = text_list[1].split(',')[0]
        text_label_2 = text_list[2].split(',')[0]
        text_label_3 = text_list[3].split(',')[0]
        if text_label_1 == text_label_2:
            text_label_result = text_label_1
        elif text_label_1 == text_label_3:
            text_label_result = text_label_1
        elif text_label_2 == text_label_3:
            text_label_result = text_label_2

        image_label_result = None
        image_label_1 = text_list[1].split(',')[1]
        image_label_2 = text_list[2].split(',')[1]
        image_label_3 = text_list[3].split(',')[1]
        if image_label_1 == image_label_2:
            image_label_result = image_label_1
        elif image_label_1 == image_label_3:
            image_label_result = image_label_1
        elif image_label_2 == image_label_3:
            image_label_result = image_label_2

        if not ((text_label_result=='positive' and image_label_result=='negative') or (text_label_result=='positive' and image_label_result=='negative')
                or image_label_result is None or text_label_result is None):
            ids.append(text_list[0])
            texts.append(text_label_result)
            images.append(image_label_result)
ids.pop(0)
texts.pop(0)
images.pop(0)

'image'

In [7]:
import pandas as pd
dataset = {'ID':ids, 'Text':texts, 'Image':images}
df = pd.DataFrame(dataset)
df.to_csv('output_multi.csv', index=False)
df

,ID,Text,Image
0,2499,positive,positive
1,2500,neutral,positive
2,2502,positive,positive
3,2504,positive,neutral
4,2505,positive,neutral
...,...,...,...
17126,22885,positive,positive
17127,22886,neutral,neutral
17128,22888,positive,neutral
17129,22889,positive,positive


In [8]:
import random
import shutil

def oversample_minority_classes(df, folder):
    class_counts = df['Image'].value_counts()
    max_count = class_counts.max()

    for cls, count in class_counts.items():
        if count < max_count:
            cls_df = df[df['Image'] == cls]
            num_samples_needed = round(max_count*0.7) - count

            for i in range(num_samples_needed):
                row = cls_df.sample().iloc[0]
                original_image_path = os.path.join(folder, row['Image'], f"{row['ID']}.jpg")
                new_image_id = f"{row['ID']}_oversampled_{i}"
                new_image_path = os.path.join(folder, row['Image'], f"{new_image_id}.jpg")
                shutil.copy(original_image_path, new_image_path)
                new_row = row.copy()
                new_row['ID'] = new_image_id
                df = df.append(new_row, ignore_index=True)

    return df

In [9]:
import os
import shutil
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def split_balanced_images(image_folder, dataset_csv, train_folder, val_folder, test_folder, ratio=(8, 1, 1)):
    # Read the dataset CSV file
    df = pd.read_csv(dataset_csv)

    # Shuffle the dataset
    df = df.sample(frac=1).reset_index(drop=True)

    # Create the output folders if they don't exist
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    unique_classes = df['Image'].unique()

    # Create class subfolders in train, val, and test folders
    for cls in unique_classes:
        os.makedirs(os.path.join(train_folder, cls), exist_ok=True)
        os.makedirs(os.path.join(val_folder, cls), exist_ok=True)
        os.makedirs(os.path.join(test_folder, cls), exist_ok=True)
    
    train_val_ratio = (ratio[0] + ratio[1]) / sum(ratio)
    train_ratio = ratio[0] / (ratio[0] + ratio[1])

    train_df_final = pd.DataFrame()
    val_df_final = pd.DataFrame()
    test_df_final = pd.DataFrame()
    # Perform stratified sampling and move the images to the corresponding class subfolders
    for cls in unique_classes:
        class_df = df[df['Image'] == cls]
        train_val_df, test_df = train_test_split(class_df, test_size=1 - train_val_ratio, stratify=class_df['Image'], random_state=42)
        train_df, val_df = train_test_split(train_val_df, test_size=1 - train_ratio, stratify=train_val_df['Image'], random_state=42)

        for _, row in train_df.iterrows():
            shutil.move(os.path.join(image_folder, f"{row['ID']}.jpg"), os.path.join(train_folder, row['Image'], f"{row['ID']}.jpg"))
        for _, row in val_df.iterrows():
            shutil.move(os.path.join(image_folder, f"{row['ID']}.jpg"), os.path.join(val_folder, row['Image'], f"{row['ID']}.jpg"))
        for _, row in test_df.iterrows():
            shutil.move(os.path.join(image_folder, f"{row['ID']}.jpg"), os.path.join(test_folder, row['Image'], f"{row['ID']}.jpg"))
        train_df_final = pd.concat([train_df_final, train_df], axis=0)
        val_df_final = pd.concat([val_df_final, val_df], axis=0)
        test_df_final = pd.concat([test_df_final, test_df], axis=0)

    # Oversample the minority classes in the train, validation, and test sets
    train_df = oversample_minority_classes(train_df_final, train_folder)
    val_df = oversample_minority_classes(val_df_final, val_folder)
    test_df = oversample_minority_classes(test_df_final, test_folder)

# Split the images
split_balanced_images(image_folder, dataset_csv, train_folder, val_folder, test_folder)

C:\Users\Jeffan\AppData\Local\Temp\ipykernel_11004\1524181078.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\Jeffan\AppData\Local\Temp\ipykernel_11004\1524181078.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\Jeffan\AppData\Local\Temp\ipykernel_11004\1524181078.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [15]:
from PIL import Image
def is_image_valid(image_path):
    try:
        with open(image_path, 'rb') as f:
            img = Image.open(f)
            img.verify()
        return True
    except Exception as e:
        print(f"Skipping invalid image {image_path}: {e}")
        return False

In [16]:
def split_balanced_images(image_folder, dataset_csv, train_folder, val_folder, test_folder, ratio=(8, 1, 1)):
    # Read the dataset CSV file
    df = pd.read_csv(dataset_csv)

    # Shuffle the dataset
    df = df.sample(frac=1).reset_index(drop=True)

    # Split the dataset into train, val, and test sets
    train_count = int(len(df) * ratio[0] / sum(ratio))
    val_count = int(len(df) * ratio[1] / sum(ratio))
    test_count = len(df) - train_count - val_count

    train_df = df.iloc[:train_count]
    val_df = df.iloc[train_count:train_count + val_count]
    test_df = df.iloc[train_count + val_count:]

    # Create the output folders if they don't exist
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    # Get the unique classes
    unique_classes = df['Image'].unique()

    # Create class subfolders in train, val, and test folders
    for cls in unique_classes:
        os.makedirs(os.path.join(train_folder, cls), exist_ok=True)
        os.makedirs(os.path.join(val_folder, cls), exist_ok=True)
        os.makedirs(os.path.join(test_folder, cls), exist_ok=True)

    # Move the images to the corresponding class subfolders
    for _, row in train_df.iterrows():
        src = os.path.join(image_folder, f"{row['ID']}.jpg")
        if is_image_valid(src):
            shutil.move(src, os.path.join(train_folder, row['Image'], f"{row['ID']}.jpg"))
        # shutil.move(os.path.join(image_folder, f"{row['ID']}.jpg"), os.path.join(train_folder, row['Image'], f"{row['ID']}.jpg"))
    for _, row in val_df.iterrows():
        src = os.path.join(image_folder, f"{row['ID']}.jpg")
        if is_image_valid(src):
            shutil.move(src, os.path.join(val_folder, row['Image'], f"{row['ID']}.jpg"))
        # shutil.move(os.path.join(image_folder, f"{row['ID']}.jpg"), os.path.join(val_folder, row['Image'], f"{row['ID']}.jpg"))
    for _, row in test_df.iterrows():
        src = os.path.join(image_folder, f"{row['ID']}.jpg")
        if is_image_valid(src):
            shutil.move(src, os.path.join(test_folder, row['Image'], f"{row['ID']}.jpg"))
        # shutil.move(os.path.join(image_folder, f"{row['ID']}.jpg"), os.path.join(test_folder, row['Image'], f"{row['ID']}.jpg"))

split_balanced_images(image_folder, dataset_csv, train_folder, val_folder, test_folder)


Skipping invalid image C:\Users\Jeffan\Downloads\MVSA-multiple\MVSA\data\5995.jpg: cannot identify image file <_io.BufferedReader name='C:\\Users\\Jeffan\\Downloads\\MVSA-multiple\\MVSA\\data\\5995.jpg'>
Skipping invalid image C:\Users\Jeffan\Downloads\MVSA-multiple\MVSA\data\3910.jpg: cannot identify image file <_io.BufferedReader name='C:\\Users\\Jeffan\\Downloads\\MVSA-multiple\\MVSA\\data\\3910.jpg'>


In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    directory= './MVSA/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 13702 images belonging to 3 classes.


In [5]:
import pandas as pd
pd.Index(train_generator.classes).value_counts()

2    7697
1    5177
0     828
dtype: int64